# TurboSL: Neural Inverse Structured Light - Google Colab Training

这个 notebook 用于在 Google Colab 上训练 TurboSL 模型

**重要**: 确保在 Colab 中选择 GPU 运行时
- Runtime -> Change runtime type -> Hardware accelerator -> GPU

## 1. 检查 GPU 环境

In [ ]:
import torch
import sys

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("WARNING: No GPU detected! Please enable GPU in Runtime settings.")

## 2. 克隆项目仓库

In [ ]:
# 克隆项目
!git clone https://github.com/guge2/TurboSL.git
%cd TurboSL

# 查看项目结构
!ls -la

## 3. 安装依赖

### 3.1 安装 PyTorch (如果需要特定版本)

In [ ]:
# Colab 通常已预装 PyTorch，但如果需要特定版本，可以取消注释下面的命令
# !pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --index-url https://download.pytorch.org/whl/cu117

# 检查当前 PyTorch 版本
import torch
print(f"Current PyTorch: {torch.__version__}")
print(f"CUDA: {torch.version.cuda}")

### 3.2 安装 tiny-cuda-nn (关键依赖)

In [ ]:
# 安装编译工具
!apt-get update
!apt-get install -y build-essential

# 安装 tiny-cuda-nn
print("Installing tiny-cuda-nn... This may take 5-10 minutes...")
!pip install git+https://github.com/NVlabs/tiny-cuda-nn/@212104156403bd87616c1a4f73a1c5f2c2e172a9#subdirectory=bindings/torch
print("tiny-cuda-nn installation completed!")

### 3.3 安装其他依赖

In [ ]:
# 安装项目依赖
# 注意: 某些包可能与 Colab 预装版本冲突，这是正常的
!pip install -r code/requirements.txt --quiet

print("All dependencies installed!")

## 4. 检查数据

In [ ]:
import os

# 检查数据目录
data_dir = "data/captured_data/david/david_opt0_n3_f128"

if os.path.exists(data_dir):
    print(f"✓ Data directory found: {data_dir}")
    
    # 列出数据内容
    print("\nData structure:")
    !ls -lh {data_dir}
    
    print("\nImages:")
    !ls -lh {data_dir}/images/scene_images/
    
    print("\nPatterns:")
    !ls -lh {data_dir}/patterns/
else:
    print(f"✗ Data directory not found: {data_dir}")
    print("Please check if the data is included in the repository.")

## 5. 配置训练参数 (可选)

In [ ]:
# 查看当前配置
!cat code/configs/neus-sl-geoalbamb-capture-blur.yaml

### 5.1 创建快速测试配置 (可选)

如果想快速测试，可以创建一个训练步数较少的配置

In [ ]:
# 可选: 创建快速测试配置
# 取消注释下面的代码来使用快速配置

# import yaml
# with open('code/configs/neus-sl-geoalbamb-capture-blur.yaml', 'r') as f:
#     config = yaml.safe_load(f)
# 
# # 修改训练步数为 2000 (快速测试)
# config['trainer']['max_steps'] = 2000
# config['trainer']['val_check_interval'] = 500
# 
# # 保存为新配置
# with open('code/configs/neus-sl-quick-test.yaml', 'w') as f:
#     yaml.dump(config, f)
# 
# print("Quick test config created!")
# config_file = "configs/neus-sl-quick-test.yaml"

# 使用默认配置
config_file = "configs/neus-sl-geoalbamb-capture-blur.yaml"
print(f"Using config: {config_file}")

## 6. 开始训练

In [ ]:
# 切换到 code 目录
%cd code

# 开始训练
!python launch.py --config {config_file} --gpu 0

## 7. 监控训练进度

训练过程中，你可以在另一个 cell 中查看日志

In [ ]:
# 查看最新的实验目录
!ls -lt exp/

In [ ]:
# 查看训练日志 (替换为实际的实验名称)
# !tail -100 exp/[experiment_name]/csv_logs/metrics.csv

## 8. 可视化训练结果

In [ ]:
# 安装 TensorBoard
%load_ext tensorboard

# 启动 TensorBoard (替换为实际的实验名称)
# %tensorboard --logdir runs/

## 9. 下载训练结果

In [ ]:
# 压缩训练结果
import os
from datetime import datetime

# 查找最新的实验目录
exp_dirs = sorted([d for d in os.listdir('exp') if os.path.isdir(os.path.join('exp', d))], 
                  key=lambda x: os.path.getmtime(os.path.join('exp', x)), 
                  reverse=True)

if exp_dirs:
    latest_exp = exp_dirs[0]
    print(f"Latest experiment: {latest_exp}")
    
    # 压缩结果
    output_name = f"turbosl_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.zip"
    !cd exp && zip -r ../{output_name} {latest_exp}
    
    print(f"\nResults saved to: {output_name}")
    print("You can download it from the file browser on the left.")
    
    # 或者直接下载
    from google.colab import files
    files.download(output_name)
else:
    print("No experiment found.")

## 10. 故障排查

### 常见问题:

1. **GPU 不可用**
   - 检查: Runtime -> Change runtime type -> Hardware accelerator -> GPU

2. **tiny-cuda-nn 安装失败**
   - 确保 CUDA 版本正确
   - 尝试重新运行安装命令

3. **显存不足 (OOM)**
   - 减少 `train_num_rays` (256 -> 128)
   - 减少 `num_samples_per_ray` (1024 -> 512)

4. **会话超时**
   - Colab 免费版有时间限制
   - 定期保存检查点
   - 考虑使用 Colab Pro

## 附录: 系统信息

In [ ]:
# 查看完整的系统信息
!nvidia-smi
!nvcc --version